In [1]:
import pdfplumber
import re  # Importation de l'expression régulière
import os
import shutil
import stat

# Dossiers à comparer
lieferschein_folder = "D:/Work_on_Benz/Lieferung_Benz"
rechnung_folder = "D:/Work_on_Benz/Rechnung_Benz"
result_folder = "D:/Work_on_Benz/Result_folder"

# Modifier les permissions pour donner un accès complet
os.chmod(lieferschein_folder, stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO),
os.chmod(rechnung_folder, stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO),
os.chmod(result_folder, stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO),

# Fonction pour extraire la Belegnummer avec pdfplumber
def extract_belegnummer(file_path):
    belegnummer = ""
    
    # Ouvrir le fichier PDF avec pdfplumber
    with pdfplumber.open(file_path) as pdf:
        # Parcourir chaque page
        for page in pdf.pages:
            text = page.extract_text()
            
            # Recherche de la Belegnummer dans le texte extrait
            if "Beleg-Nr." in text:
                belegnummer = extract_data(text, "Beleg-Nr.")
    
    # Filtrer la valeur pour ne garder que les chiffres
    belegnummer = clean_value(belegnummer)
    
    return belegnummer


In [2]:
# Fonction pour extraire la Lieferschein avec pdfplumber
def extract_lieferschein(file_path):
    lieferschein = ""
    
    # Ouvrir le fichier PDF avec pdfplumber
    with pdfplumber.open(file_path) as pdf:
        # Parcourir chaque page
        for page in pdf.pages:
            text = page.extract_text()
            
            # Recherche de la Lieferschein dans le texte extrait
            if "Lieferschein" in text:
                lieferschein = extract_data(text, "Lieferschein")
    
    # Filtrer la valeur pour ne garder que les chiffres
    lieferschein = clean_value(lieferschein)
    
    return lieferschein

In [8]:
extract_lieferschein(lieferschein_folder)

PermissionError: [Errno 13] Permission denied: 'D:/Work_on_Benz/Lieferung_Benz'

In [4]:
# Fonction pour extraire la donnée après une étiquette donnée
def extract_data(text, label):
    start_index = text.find(label)
    if start_index == -1:
        return None
    start_index += len(label)
    end_index = text.find("\n", start_index)
    return text[start_index:end_index].strip()

In [5]:
# Fonction pour nettoyer les valeurs extraites en ne conservant que les chiffres
def clean_value(value):
    # Utilisation de l'expression régulière pour ne garder que les chiffres
    if value:
        cleaned_value = re.sub(r'\D', '', value)  # Remplacer tout ce qui n'est pas un chiffre par ""
        return cleaned_value[:9]  # Limiter à 9 chiffres pour la Belegnummer
    return ""


In [7]:
# Fonction pour comparer les fichiers de deux dossiers
def compare_and_move_files(lieferschein_folder, rechnung_folder, result_folder):
    # Créer le dossier 'matched' s'il n'existe pas
    matched_folder = os.path.join(result_folder, 'matched')
    os.makedirs(matched_folder, exist_ok=True)

    # Créer le dossier 'unmatched' s'il n'existe pas
    unmatched_folder = os.path.join(result_folder, 'unmatched')
    os.makedirs(unmatched_folder, exist_ok=True)

    # Parcourir tous les fichiers dans le dossier Lieferschein
    for lieferschein_file in os.listdir(lieferschein_folder):
        lieferschein_path = os.path.join(lieferschein_folder, lieferschein_file)
        
        if not lieferschein_file.endswith('.pdf'):
            continue

        # Extraire la Belegnummer du fichier Lieferschein
        belegnummer_lieferschein = extract_belegnummer(lieferschein_path)
        print(f"Belegnummer extrait du fichier {lieferschein_file}: {belegnummer_lieferschein}")
        
        # Comparer avec les fichiers du dossier Rechnung
        for rechnung_file in os.listdir(rechnung_folder):
            rechnung_path = os.path.join(rechnung_folder, rechnung_file)
            
            if not rechnung_file.endswith('.pdf'):
                continue

            # Extraire la Lieferschein du fichier Rechnung
            lieferschein_rechnung = extract_lieferschein(rechnung_path)
            print(f"Lieferschein extrait du fichier {rechnung_file}: {lieferschein_rechnung}")
            
            # Comparaison des Belegnummer
            if belegnummer_lieferschein == lieferschein_rechnung:
                print(f"Match trouvé: {lieferschein_file} - {rechnung_file}")

                # Créer un sous-dossier avec le numéro de matching
                match_folder = os.path.join(matched_folder, belegnummer_lieferschein)
                os.makedirs(match_folder, exist_ok=True)

                # Créer les sous-dossiers "Bon de Livraison" et "Facture"
                bon_de_livraison_folder = os.path.join(match_folder, "Bon de Livraison")
                facture_folder = os.path.join(match_folder, "Facture")

                os.makedirs(bon_de_livraison_folder, exist_ok=True)
                os.makedirs(facture_folder, exist_ok=True)

                # Déplacer les fichiers dans les sous-dossiers respectifs
                shutil.move(lieferschein_path, os.path.join(bon_de_livraison_folder, lieferschein_file))
                shutil.move(rechnung_path, os.path.join(facture_folder, rechnung_file))

            else:
                print(f"No match: {lieferschein_file} - {rechnung_file}")
                # Déplacer les fichiers non correspondants dans le dossier 'unmatched'
                shutil.move(lieferschein_path, os.path.join(unmatched_folder, lieferschein_file))
                shutil.move(rechnung_path, os.path.join(unmatched_folder, rechnung_file))



# Exécution de la comparaison
compare_and_move_files(lieferschein_folder, rechnung_folder, result_folder)
